In [ ]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import itertools
import math
import os
import threading
import concurrent.futures
import numba as nb
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import wait
from concurrent.futures import as_completed
from joblib import Parallel, delayed
from numba import njit

In [ ]:
# Load in data
df = pd.read_csv('cleaned_bullying.csv')

# Drop columns
df = df.drop(columns=['Unnamed: 0',
                      'Bullied_on_school_property_in_past_12_months',
                      'Bullied_not_on_school_property_in_past_12_months',
                      'Cyber_bullied_in_past_12_months'])

# Split into X and y
X = df.drop('bullied', axis=1).values
y = df['bullied'].values

# Load copy of dataframe
data = df

# Train/test split
train_data = data.sample(frac=0.7, random_state=1)
test_data = data.drop(train_data.index)

# 1. Decision Tree Model from Scratch

In [ ]:
def generate_possible_splits(data):
    # generate all possible binary splits of the data
    features = list(data.columns)[:-1]
    splits = []
    for feature in features:
        values = data[feature].unique()
        for i in range(1, len(values)):
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
    return splits

def evaluate_split(split, data):
    # evaluate a split by calculating the information gain
    feature, values = split
    total = len(data)
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    entropy_before = calculate_entropy(data)
    entropy_after = (left_proportion * calculate_entropy(left) +
                     right_proportion * calculate_entropy(right))
    information_gain = entropy_before - entropy_after
    return information_gain, left, right

def calculate_entropy(data):
    # calculate the entropy of a set of data
    total = len(data)
    counts = data.iloc[:, -1].value_counts()
    entropy = 0
    for count in counts:
        proportion = count / total
        entropy -= proportion * math.log2(proportion)
    return entropy

def build_decision_tree(data):
    # build a decision tree using a brute force algorithm
    if len(data.iloc[:, -1].unique()) == 1:
        # all instances have the same class
        return {'class': data.iloc[0, -1]}
    best_split = None
    best_gain = 0
    for split in generate_possible_splits(data):
        gain, left, right = evaluate_split(split, data)
        if gain > best_gain:
            best_gain = gain
            best_split = split
            best_left = left
            best_right = right
    if best_split is None:
        # no split improved information gain
        return {'class': data.iloc[:, -1].value_counts().idxmax()}
    else:
        left_subtree = build_decision_tree(best_left)
        right_subtree = build_decision_tree(best_right)
        return {'feature': best_split[0],
                'value': best_split[1],
                'left': left_subtree,
                'right': right_subtree}

def predict(instance, tree):
    if 'class' in tree:
        # leaf node, return class
        return tree['class']
    else:
        feature = tree['feature']
        value = tree['value']
        if instance[feature] in value:
            subtree = tree['left']
        else:
            subtree = tree['right']
        return predict(instance, subtree)
    
def evaluate():
    # load dataset
    data = df

    # split data into training and test sets
    train_data = data.sample(frac=0.7, random_state=1)
    test_data = data.drop(train_data.index)

    # train decision tree
    start = time.time()
    tree = build_decision_tree(train_data)
    end = time.time()
    time_diff = end - start

    # evaluate decision tree
    y_true = test_data.iloc[:, -1]
    y_pred = test_data.apply(lambda x: predict(x, tree), axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred))

    # print results
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Time: {time_diff}')
    
    # Make beeping sounds to notify end of evaluation
    beep = lambda x: os.system("echo '\a';sleep 0.5;" * x)
    beep(10)

In [ ]:
evaluate()

AUC: 0.5807639826418268
Accuracy: 0.6037850419997975
Time: 273.81109714508057












# 2. Evalute Decision Tree Model Using Line Profiler

~96% of time spent on evaluate_split(split, data)

In [ ]:
%load_ext line_profiler

In [ ]:
%lprun -f build_decision_tree tree = build_decision_tree(train_data)

# 3. Optimize Decision Tree with Algorithm Changes

- Avoid calculating info gain for all possible splits (brute force)
- Instead, choose random subset of features to split on (reducing number of splits that need to be evaluted)
- Implemented by changing 2 functions: 
    - generate_possible_splits
    - evaluate_split

In [ ]:
## NEW: generate random binary splits of the data ###
# Randomly sample a subset of the features to split on #
# This reduces the number of splits that need to be evaluated, #
# but still allows the algorithm to find a diverse range of splits. #
def generate_possible_splits(data, num_features, num_values):
    features = list(data.columns)[:-1]
    # WITH THIS LINE BELOW
    features = list(np.random.choice(features, num_features, replace=False))
    splits = []
    for feature in features:
        values = data[feature].unique()
        values = list(np.random.choice(values, min(num_values, len(values)), replace=False))
        for i in range(1, len(values)):
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
    return splits

# Avoid calculating the information gain for all evaluation splits #
def evaluate_split(split, data):
    # evaluate a split by calculating the information gain
    feature, values = split
    total = len(data)
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    # WITH THIS LINE BELOW #
    if len(left) == 0 or len(right) == 0:
        return -float('inf'), None, None
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    entropy_before = calculate_entropy(data)
    entropy_after = (left_proportion * calculate_entropy(left) +
                     right_proportion * calculate_entropy(right))
    information_gain = entropy_before - entropy_after
    return information_gain, left, right

# Update calls to generate_possible_splits to have the 
# correct inputs (adding num_features and num_values)
def build_decision_tree(data):
    # build a decision tree using a brute force algorithm
    if len(data.iloc[:, -1].unique()) == 1:
        # all instances have the same class
        return {'class': data.iloc[0, -1]}
    best_split = None
    best_gain = 0
    for split in generate_possible_splits(data, num_features=5, num_values=3):
        gain, left, right = evaluate_split(split, data)
        if gain > best_gain:
            best_gain = gain
            best_split = split
            best_left = left
            best_right = right
    if best_split is None:
        # no split improved information gain
        return {'class': data.iloc[:, -1].value_counts().idxmax()}
    else:
        left_subtree = build_decision_tree(best_left)
        right_subtree = build_decision_tree(best_right)
        return {'feature': best_split[0],
                'value': best_split[1],
                'left': left_subtree,
                'right': right_subtree}

In [ ]:
evaluate()

AUC: 0.5964579690801155
Accuracy: 0.6250379516243295
Time: 56.65168905258179












# 4. Optimize Decision Trees with Threading for evaluate_split Function

In [ ]:
def evaluate_split(split, data, num_threads=4):
    # evaluate a split by calculating the information gain
    feature, values = split
    total = len(data)
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    # WITH THIS LINE BELOW #
    if len(left) == 0 or len(right) == 0:
        return -float('inf'), None, None
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(calculate_entropy, subset) for subset in [data, left, right]]
        results = [f.result() for f in futures]
    
    entropy_before = results[0]
    entropy_after = (left_proportion * results[1] +
                     right_proportion * results[2])
    information_gain = entropy_before - entropy_after
    return information_gain, left, right

In [ ]:
evaluate()

AUC: 0.5960389750233661
Accuracy: 0.622305434672604
Time: 70.43838691711426












# 5. Optimizing Model by Adding Variables for Number of Features and Number of Values

In [ ]:
def generate_possible_splits(data, num_features, num_values):
    features = list(data.columns)[:-1]
    # WITH THIS LINE BELOW
    features = list(np.random.choice(features, num_features, replace=False))
    splits = []
    for feature in features:
        values = data[feature].unique()
        values = list(np.random.choice(values, min(num_values, len(values)), replace=False))
        for i in range(1, len(values)):
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
    return splits

# Avoid calculating the information gain for all evaluation splits
def evaluate_split(split, data):
    # evaluate a split by calculating the information gain
    feature, values = split
    total = len(data)
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    if len(left) == 0 or len(right) == 0:
        return None
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    entropy_before = calculate_entropy(data)
    entropy_after = (left_proportion * calculate_entropy(left) +
                     right_proportion * calculate_entropy(right))
    information_gain = entropy_before - entropy_after
    if information_gain <= 0:
        return None
    return information_gain, left, right, split


def calculate_entropy(data):
    # calculate the entropy of a set of data
    total = len(data)
    counts = data.iloc[:, -1].value_counts()
    entropy = 0
    for count in counts:
        proportion = count / total
        entropy -= proportion * math.log2(proportion)
    return entropy


# Update calls to generate_possible_splits to have the 
# correct inputs (adding num_features and num_values)
def build_decision_tree(data, num_features, num_values):
    # build a decision tree using a brute force algorithm
    if len(data.iloc[:, -1].unique()) == 1:
        # all instances have the same class
        return {'class': data.iloc[0, -1]}
    splits = generate_possible_splits(data, num_features, num_values)
    splits_gain = [split_gain for split_gain in (evaluate_split(split, data) for split in splits) if split_gain]
    if not splits_gain:
        return {'class': data.iloc[:, -1].value_counts().idxmax()}
    best_gain, best_left, best_right, best_split = max(splits_gain, key=lambda x: x[0])
    left_subtree = build_decision_tree(best_left, num_features, num_values)
    right_subtree = build_decision_tree(best_right, num_features, num_values)
    return {'feature': best_split[0],
            'value': best_split[1],
            'left': left_subtree,
            'right': right_subtree}


def predict(instance, tree):
    if 'class' in tree:
        # leaf node, return class
        return tree['class']
    else:
        feature = tree['feature']
        value = tree['value']
        if instance[feature] in value:
            subtree = tree['left']
        else:
            subtree = tree['right']
        return predict(instance, subtree)
    
def evaluate():
    # load dataset
    data = df

    # split data into training and test sets
    train_data = data.sample(frac=0.7, random_state=1)
    test_data = data.drop(train_data.index)

    # train decision tree
    start = time.time()
    tree = build_decision_tree(train_data, num_features=3, num_values=3)
    end = time.time()
    time_diff = end - start

    # evaluate decision tree
    y_true = test_data.iloc[:, -1]
    y_pred = test_data.apply(lambda x: predict(x, tree), axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred))

    # print results
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Time: {time_diff}')
    
    # Make beeping sounds to notify end of evaluation
    beep = lambda x: os.system("echo '\a';sleep 0.5;" * x)
    beep(10)

In [ ]:
evaluate()

AUC: 0.6079748258819964
Accuracy: 0.6395101710353203
Time: 25.417531967163086










